In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1rc6.dev101
cdv, version 1.1.2
Python 3.10.8


## DID puzzle
- [DIDs](https://chialisp.com/dids/)
> DIDs are a decentralized way to represent an identity, be that an organization or a person. It is possible to recover a DID even if the key is lost, by having multiple parties send messages approving the inner puzzle change. More functionality will be added to the DID standard in the future, allowing for easier identification and recovery methods.

- [chia-blockchain/chia/wallet/puzzles/did_innerpuz.clvm](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/did_innerpuz.clvm)
> ; The DID innerpuzzle is designed to sit inside the singleton layer and provide functionality related to being an identity.
; At the moment the two pieces of functionality are recovery and message creation.
; A DID's ID is it's Singleton ID
; Recovery is based around having a list of known other DIDs which can send messages approving you change the innerpuzzle of your DID singleton


In [2]:
%%bash
chia keys derive -f 3855208971 child-key -t wallet -i 0 --show-hd-path -s
chia keys derive -f 3855208971 wallet-address -i 0 --show-hd-path

Wallet public key 0 (m/12381/8444/2/0): b87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84
Wallet private key 0 (m/12381/8444/2/0): 31d87cd97f07d1b44064254617adc76a54d4fc176a557e7d8760b0a1c7f050f9
Wallet address 0 (m/12381/8444/2/0): txch1s705m30vt5cvdd4wex59x8fwcgym6cvd49n6ffrzmm8vlyvycansfvjfp9


In [3]:
alice_fp = 3855208971
alice_addr = "txch1s705m30vt5cvdd4wex59x8fwcgym6cvd49n6ffrzmm8vlyvycansfvjfp9"
alice_ph = decode_puzzle_hash(alice_addr)
alice_sk = PrivateKey.from_bytes(bytes.fromhex("31d87cd97f07d1b44064254617adc76a54d4fc176a557e7d8760b0a1c7f050f9"))
alice_pk = alice_sk.get_g1()

print("Alice:")
print(f"\tfingerprint:\t{alice_fp}")
print(f"\taddress:\t{alice_addr}")
print(f"\tsk:\t\t{bytes(alice_sk).hex()}")
print(f"\tpk:\t\t{alice_pk}")

Alice:
	fingerprint:	3855208971
	address:	txch1s705m30vt5cvdd4wex59x8fwcgym6cvd49n6ffrzmm8vlyvycansfvjfp9
	sk:		31d87cd97f07d1b44064254617adc76a54d4fc176a557e7d8760b0a1c7f050f9
	pk:		b87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84


In [4]:
%%bash
chia keys derive -f 4174968354 child-key -t wallet -i 0 --show-hd-path -s
chia keys derive -f 4174968354 wallet-address -i 0 --show-hd-path

Wallet public key 0 (m/12381/8444/2/0): 831c75d8db815b7e1972d99b238cb62d9ba23b8668143b8c1de237d4a504f745277312917e38e9bdf9bec60b3615de3a
Wallet private key 0 (m/12381/8444/2/0): 156f2ae00d0506a166031a999915a64b608f733ec068a4ee85962c187edeeb02
Wallet address 0 (m/12381/8444/2/0): txch1xquze68sl2lkdeahve3zew44xhhtdk6uqms5nnpm5mx04am4k0wsa363eq


In [5]:
bob_fp = 4174968354
bob_addr = "txch1xquze68sl2lkdeahve3zew44xhhtdk6uqms5nnpm5mx04am4k0wsa363eq"
bob_ph = decode_puzzle_hash(alice_addr)
bob_sk = PrivateKey.from_bytes(bytes.fromhex("156f2ae00d0506a166031a999915a64b608f733ec068a4ee85962c187edeeb02"))
bob_pk = bob_sk.get_g1()

print("Bob:")
print(f"\tfingerprint:\t{bob_fp}")
print(f"\taddress:\t{bob_addr}")
print(f"\tsk:\t\t{bytes(bob_sk).hex()}")
print(f"\tpk:\t\t{bob_pk}")

Bob:
	fingerprint:	4174968354
	address:	txch1xquze68sl2lkdeahve3zew44xhhtdk6uqms5nnpm5mx04am4k0wsa363eq
	sk:		156f2ae00d0506a166031a999915a64b608f733ec068a4ee85962c187edeeb02
	pk:		831c75d8db815b7e1972d99b238cb62d9ba23b8668143b8c1de237d4a504f745277312917e38e9bdf9bec60b3615de3a


### Create DID wallet
- [RPC](https://docs.chia.net/did-rpc#create_new_wallet)
- [API](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/rpc/wallet_rpc_api.py#L542)

In [6]:
wallet_client = await get_synced_wallet_client(alice_fp)

# create DID for alice
response = await wallet_client.create_new_did_wallet(
        amount = 1,
        fee = 0,
        name = "DID Wallet"
)
print(response)

await close_rpc_client(wallet_client)

fingerprint: 3855208971
False
{'my_did': 'did:chia:10hq8mukapz8lml97tgnnsfdv24t9u5mw6pnqfh64kwh6vljykyrsc4m38t', 'success': True, 'type': 8, 'wallet_id': 2}


In [8]:
%%bash
chia rpc wallet did_get_current_coin_info '{"wallet_id": 2}'
chia rpc wallet did_get_did '{"wallet_id": 2}'

{
    "did_amount": 1,
    "did_innerpuz": "0x3b28694e28afea944de7a5af588685ad7a30887f97cdebf719fc8b368dae85c6",
    "did_parent": "0x3ea1c0ec5d2be2c8b6d9b70218f8169d33735775d5edbc651316164cab6983c0",
    "my_did": "did:chia:10hq8mukapz8lml97tgnnsfdv24t9u5mw6pnqfh64kwh6vljykyrsc4m38t",
    "success": true,
    "wallet_id": 2
}
{
    "coin_id": "0x8ef7c734bcc2789cc13d7766423733308a92f5a649476829694d646a34d13776",
    "my_did": "did:chia:10hq8mukapz8lml97tgnnsfdv24t9u5mw6pnqfh64kwh6vljykyrsc4m38t",
    "success": true,
    "wallet_id": 2
}


In [9]:
# did is the encoded launcher_id
did = "did:chia:10hq8mukapz8lml97tgnnsfdv24t9u5mw6pnqfh64kwh6vljykyrsc4m38t"
launcher_id = decode_puzzle_hash(did)

full_node_client = await get_full_node_client()
print(f"launcher_id: {launcher_id}")

# get launcher_coin
coin_record = await full_node_client.get_coin_record_by_name(launcher_id)
spent_height = coin_record.spent_block_index
print(coin_record)
print()

# get puzzle and solution
puzzle_solution = await full_node_client.get_puzzle_and_solution(launcher_id, spent_height)
puzzle = puzzle_solution.puzzle_reveal.to_program()
solution = puzzle_solution.solution.to_program()

# (singleton_full_puzzle_hash amount key_value_list)
did_full_puzzle_hash = hexstr_to_bytes(disassemble(solution.first()))
print(f"DID PH: {did_full_puzzle_hash.hex()}")
print()

did_coin_records = await full_node_client.get_coin_records_by_puzzle_hash(did_full_puzzle_hash, include_spent_coins = True)
for cr in did_coin_records:
    print(cr.coin)
    if cr.spent_block_index > 0:
        puzzle_solution = await full_node_client.get_puzzle_and_solution(cr.coin.name(), cr.spent_block_index)
        #print(puzzle_solution.puzzle_reveal)
        puzzle = puzzle_solution.puzzle_reveal.to_program()
        solution = puzzle_solution.solution.to_program()
        print_program(puzzle)
        print()
        print_program(solution)
    print()

await close_rpc_client(full_node_client)


launcher_id: 7dc07df2dd088ffdfcbe5a273825ac55565e536ed06604df55b3afa67e44b107
{'coin': {'amount': 1,
          'parent_coin_info': '0x389e85e5bc97d408d0aad44e754490fccb047326361697173732e42f8cb6300f',
          'puzzle_hash': '0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9'},
 'coinbase': False,
 'confirmed_block_index': 6,
 'spent_block_index': 6,
 'timestamp': 1667807290}

DID PH: 4c9257a1982cb878544c7cf400d79c30131e59cdbbc66f74da2cc7a8141408e6

Coin { parent_coin_info: 3ea1c0ec5d2be2c8b6d9b70218f8169d33735775d5edbc651316164cab6983c0, puzzle_hash: 4c9257a1982cb878544c7cf400d79c30131e59cdbbc66f74da2cc7a8141408e6, amount: 1 }

Coin { parent_coin_info: 7dc07df2dd088ffdfcbe5a273825ac55565e536ed06604df55b3afa67e44b107, puzzle_hash: 4c9257a1982cb878544c7cf400d79c30131e59cdbbc66f74da2cc7a8141408e6, amount: 1 }
(a (q 2 (q 2 (i (logand 47 52) (q 4 (c 32 (c 47 ())) (c (a 62 (c 2 (c 5 (c (a 42 (c 2 (c 39 (c (a (i 119 (q 2 54 (c 2 (c 9 (c 87 (c (a 46 (c 2 (c 5 ()))) ()))))) (

In [36]:
spent_did_cr = did_coin_records[1]
print(spent_did_cr.coin.name().hex())
full_node_client = await get_full_node_client()
puzzle_solution = await full_node_client.get_puzzle_and_solution(spent_did_cr.coin.name(), spent_did_cr.spent_block_index)
await close_rpc_client(full_node_client)


puzzle = puzzle_solution.puzzle_reveal.to_program()
solution = puzzle_solution.solution.to_program()
print_program(puzzle)
print()
print_program(solution)

3ea1c0ec5d2be2c8b6d9b70218f8169d33735775d5edbc651316164cab6983c0
(a (q 2 (q 2 (i (logand 47 52) (q 4 (c 32 (c 47 ())) (c (a 62 (c 2 (c 5 (c (a 42 (c 2 (c 39 (c (a (i 119 (q 2 54 (c 2 (c 9 (c 87 (c (a 46 (c 2 (c 5 ()))) ()))))) (q . 29)) 1) (c (a (i 119 (q . -73) (q . 87)) 1) ()))))) (c 119 ()))))) (a 58 (c 2 (c 5 (c (a 11 95) (q ()))))))) (q 8)) 1) (c (q (((73 . 71) 2 . 51) (c . 1) 1 . 2) ((not 2 (i 5 (q 2 50 (c 2 (c 13 (c (sha256 60 (sha256 52 36) (sha256 60 (sha256 60 (sha256 52 44) 9) (sha256 60 11 (sha256 52 ())))) ())))) (q . 11)) 1) (a (i (all (= (strlen 5) 34) (= (strlen 11) 34) (> 23 (q . -1))) (q 11 5 11 23) (q 8)) 1) 2 (i 11 (q 2 (i (a 38 (c 2 (c 19 ()))) (q 2 (i (not 23) (q 2 (i (= -77 (q . -113)) (q 2 58 (c 2 (c 5 (c 27 (c 52 ()))))) (q 4 (c 35 (c (a 54 (c 2 (c 9 (c 83 (c (a 46 (c 2 (c 5 ()))) ()))))) 115)) (a 58 (c 2 (c 5 (c 27 (c 52 ()))))))) 1) (q 8)) 1) (q 4 19 (a 58 (c 2 (c 5 (c 27 (c 23 ()))))))) 1) (q 2 (i 23 () (q 8)) 1)) 1) ((a (i (= 9 56) (q 2 (i (logand 45 (q . 1

In [25]:
result = puzzle.uncurry()
uncurried_puzzle = result[0]
curried_args = result[1]

print_program(uncurried_puzzle)
print()
print_program(curried_args)

(a (q 2 (i (logand 47 52) (q 4 (c 32 (c 47 ())) (c (a 62 (c 2 (c 5 (c (a 42 (c 2 (c 39 (c (a (i 119 (q 2 54 (c 2 (c 9 (c 87 (c (a 46 (c 2 (c 5 ()))) ()))))) (q . 29)) 1) (c (a (i 119 (q . -73) (q . 87)) 1) ()))))) (c 119 ()))))) (a 58 (c 2 (c 5 (c (a 11 95) (q ()))))))) (q 8)) 1) (c (q (((73 . 71) 2 . 51) (c . 1) 1 . 2) ((not 2 (i 5 (q 2 50 (c 2 (c 13 (c (sha256 60 (sha256 52 36) (sha256 60 (sha256 60 (sha256 52 44) 9) (sha256 60 11 (sha256 52 ())))) ())))) (q . 11)) 1) (a (i (all (= (strlen 5) 34) (= (strlen 11) 34) (> 23 (q . -1))) (q 11 5 11 23) (q 8)) 1) 2 (i 11 (q 2 (i (a 38 (c 2 (c 19 ()))) (q 2 (i (not 23) (q 2 (i (= -77 (q . -113)) (q 2 58 (c 2 (c 5 (c 27 (c 52 ()))))) (q 4 (c 35 (c (a 54 (c 2 (c 9 (c 83 (c (a 46 (c 2 (c 5 ()))) ()))))) 115)) (a 58 (c 2 (c 5 (c 27 (c 52 ()))))))) 1) (q 8)) 1) (q 4 19 (a 58 (c 2 (c 5 (c 27 (c 23 ()))))))) 1) (q 2 (i 23 () (q 8)) 1)) 1) ((a (i (= 9 56) (q 2 (i (logand 45 (q . 1)) (q 1 . 1) ()) 1) ()) 1) 11 60 (sha256 52 40) (sha256 60 (sha256 60 

### Singleton Top Layer

```clojure
(SINGLETON_STRUCT INNER_PUZZLE lineage_proof my_amount inner_solution)
```

#### SINGLTON_STRUCT (SINGLETON_MOD_HASH, (LAUNCHER_ID, LAUNCHER_PUZZLE_HASH))
`(0x7faa3253bfddd1e0decb0906b2dc6247bbc4cf608f58345d173adb63e8b47c9f 0x7dc07df2dd088ffdfcbe5a273825ac55565e536ed06604df55b3afa67e44b107 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9)`

#### INNER_PUZZLE (DID_INNER_PUZZLE)
```cljoure
(a (q 2 (q 2 (i -65 (q 2 5 383) (q 2 (i (all (= (a 126 (c 2 (c 6143 ()))) 11) (> 23 ())) (q 4 (c 40 (c 383 ())) (c (c 52 (c 767 (c 383 (c (c 767 ()) ())))) (c (c 56 (c 12287 ())) (a 38 (c 2 (c 47 (c 23 (c 6143 (c 12287 (c 767 (c 1535 (c 3071 (q ()))))))))))))) (q 8)) 1)) 1) (c (q (((49 . 61) 73 . 70) (a . 51) 60 . 4) ((q . 1) 2 2 (i 5 (q 2 58 (c 2 (c 13 (c (sha256 42 (sha256 34 60) (sha256 42 (sha256 42 (sha256 34 50) 9) (sha256 42 11 (sha256 34 ())))) ())))) (q . 11)) 1) ((a (i 23 (q 2 (i 319 (q 4 (c 48 (c (sha256 (sha256 (a 54 (c 2 (c 5 (c 39 (c 575 (c 1343 (c 2879 ()))))))) (a 126 (c 2 (c (a 46 (c 2 (c 47 (c 95 (c 383 ()))))) ())))) 47) ())) (a 38 (c 2 (c 5 (c 11 (c 55 (c 47 (c 95 (c 447 (c 383 (c (+ 767 (q . 1)) ()))))))))))) (q 2 38 (c 2 (c 5 (c 55 (c 47 (c 95 (c 447 (c 383 (c 767 ())))))))))) 1) (q 2 (i (> 767 (- 11 (q . 1))) (q 4 (c 32 (c 383 (c 95 ()))) ()) (q 8)) 1)) 1) 11 23 (a 94 (c 2 (c 9 (c 47 (c (a 126 (c 2 (c (c 9 (c 11 29)) ()))) ()))))) 95) (c (q . 1) (c (c 44 (c 5 ())) (c (c 32 (c 23 (c 11 ()))) ()))) (sha256 42 (sha256 34 36) (sha256 42 (sha256 42 (sha256 34 50) 5) (sha256 42 (a 58 (c 2 (c 7 (c (sha256 34 34) ())))) (sha256 34 ())))) 2 (i (l 5) (q 11 (q . 2) (a 126 (c 2 (c 9 ()))) (a 126 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q 2 (q 2 (q 2 (i 11 (q 2 (i (= 5 (point_add 11 (pubkey_for_exp (sha256 11 (a 6 (c 2 (c 23 ()))))))) (q 2 23 47) (q 8)) 1) (q 4 (c 4 (c 5 (c (a 6 (c 2 (c 23 ()))) ()))) (a 23 47))) 1) (c (q 50 2 (i (l 5) (q 11 (q . 2) (a 6 (c 2 (c 9 ()))) (a 6 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q . 0x93c8a14b74d0cfe62cf451200eb42f3f2fe5c581f5153d308b10c43f40fba9d49a812204e0f970b2f83e23c59eb22e09) 1)) (c (q . 0x4bf5122f344554c53bde2ebb8cd2b7e3d1600ad631c385a5d7cce23c7785459a) (c (q) (c (q 0x7faa3253bfddd1e0decb0906b2dc6247bbc4cf608f58345d173adb63e8b47c9f 0x7dc07df2dd088ffdfcbe5a273825ac55565e536ed06604df55b3afa67e44b107 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) (c (q) 1))))))
```


In [30]:
singleton_struct = curried_args.first()
print_program(singleton_struct)
print()
did_inner_puzzle = curried_args.rest().first()
print_program(did_inner_puzzle)

(0x7faa3253bfddd1e0decb0906b2dc6247bbc4cf608f58345d173adb63e8b47c9f 0x7dc07df2dd088ffdfcbe5a273825ac55565e536ed06604df55b3afa67e44b107 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9)

(a (q 2 (q 2 (i -65 (q 2 5 383) (q 2 (i (all (= (a 126 (c 2 (c 6143 ()))) 11) (> 23 ())) (q 4 (c 40 (c 383 ())) (c (c 52 (c 767 (c 383 (c (c 767 ()) ())))) (c (c 56 (c 12287 ())) (a 38 (c 2 (c 47 (c 23 (c 6143 (c 12287 (c 767 (c 1535 (c 3071 (q ()))))))))))))) (q 8)) 1)) 1) (c (q (((49 . 61) 73 . 70) (a . 51) 60 . 4) ((q . 1) 2 2 (i 5 (q 2 58 (c 2 (c 13 (c (sha256 42 (sha256 34 60) (sha256 42 (sha256 42 (sha256 34 50) 9) (sha256 42 11 (sha256 34 ())))) ())))) (q . 11)) 1) ((a (i 23 (q 2 (i 319 (q 4 (c 48 (c (sha256 (sha256 (a 54 (c 2 (c 5 (c 39 (c 575 (c 1343 (c 2879 ()))))))) (a 126 (c 2 (c (a 46 (c 2 (c 47 (c 95 (c 383 ()))))) ())))) 47) ())) (a 38 (c 2 (c 5 (c 11 (c 55 (c 47 (c 95 (c 447 (c 383 (c (+ 767 (q . 1)) ()))))))))))) (q 2 38 (c 2 (c 5 (c 55 (c 47 (c 95 (c 447 (c 383 (c 7

In [31]:
result = did_inner_puzzle.uncurry()
uncurried_did_inner_puzzle = result[0]
print_program(uncurried_did_inner_puzzle)
print()
curried_args = result[1]
print_program(curried_args)

(a (q 2 (i -65 (q 2 5 383) (q 2 (i (all (= (a 126 (c 2 (c 6143 ()))) 11) (> 23 ())) (q 4 (c 40 (c 383 ())) (c (c 52 (c 767 (c 383 (c (c 767 ()) ())))) (c (c 56 (c 12287 ())) (a 38 (c 2 (c 47 (c 23 (c 6143 (c 12287 (c 767 (c 1535 (c 3071 (q ()))))))))))))) (q 8)) 1)) 1) (c (q (((49 . 61) 73 . 70) (a . 51) 60 . 4) ((q . 1) 2 2 (i 5 (q 2 58 (c 2 (c 13 (c (sha256 42 (sha256 34 60) (sha256 42 (sha256 42 (sha256 34 50) 9) (sha256 42 11 (sha256 34 ())))) ())))) (q . 11)) 1) ((a (i 23 (q 2 (i 319 (q 4 (c 48 (c (sha256 (sha256 (a 54 (c 2 (c 5 (c 39 (c 575 (c 1343 (c 2879 ()))))))) (a 126 (c 2 (c (a 46 (c 2 (c 47 (c 95 (c 383 ()))))) ())))) 47) ())) (a 38 (c 2 (c 5 (c 11 (c 55 (c 47 (c 95 (c 447 (c 383 (c (+ 767 (q . 1)) ()))))))))))) (q 2 38 (c 2 (c 5 (c 55 (c 47 (c 95 (c 447 (c 383 (c 767 ())))))))))) 1) (q 2 (i (> 767 (- 11 (q . 1))) (q 4 (c 32 (c 383 (c 95 ()))) ()) (q 8)) 1)) 1) 11 23 (a 94 (c 2 (c 9 (c 47 (c (a 126 (c 2 (c (c 9 (c 11 29)) ()))) ()))))) 95) (c (q . 1) (c (c 44 (c 5 ())) (c 

### DID_INNER_PUZZLE
```cljoure
(
  INNER_PUZZLE  ; Standard P2 inner puzzle, used to record the ownership of the DID.
  RECOVERY_DID_LIST_HASH  ; the list of DIDs that can send messages to you for recovery we store only the hash so that we don't have to reveal every time we make a message spend
  NUM_VERIFICATIONS_REQUIRED  ; how many of the above list are required for a recovery
  SINGLETON_STRUCT  ; my singleton_struct, formerly a Truth - ((SINGLETON_MOD_HASH, (LAUNCHER_ID, LAUNCHER_PUZZLE_HASH)))
  METADATA ; Customized metadata, e.g KYC info
  mode  ; this indicates which spend mode we want. 0. Recovery mode 1. Run INNER_PUZZLE with p2_solution
  my_amount_or_inner_solution  ; In mode 0, we use this to recover our coin and assert it is our actual amount
                             ; In mode 1 this is the solution of the inner P2 puzzle, only required in the create message mode and transfer mode.
  new_inner_puzhash  ; In recovery mode, this will be the new wallet DID puzzle hash
  parent_innerpuzhash_amounts_for_recovery_ids  ; during a recovery we need extra information about our recovery list coins
  pubkey  ; this is the new pubkey used for a recovery
  recovery_list_reveal  ; this is the reveal of the stored list of DIDs approved for recovery
  my_id  ; my coin ID
  )
  ;message is the new puzzle in the recovery and standard spend cases

  ;MOD_HASH, MY_PUBKEY, RECOVERY_DID_LIST_HASH are curried into the puzzle
  ;EXAMPLE SOLUTION (0xcafef00d 0x12341234 0x923bf9a7856b19d335a65f12d68957d497e1f0c16c0e14baf6d120e60753a1ce 2 1 100 (q "source code") 0xdeadbeef 0xcafef00d ((0xdadadada 0xdad5dad5 200) () (0xfafafafa 0xfaf5faf5 200)) 0xfadeddab (0x22222222 0x33333333 0x44444444))

```

#### INNER_PUZZLE with SYNTHETIC_PK
```clojure
(a (q 2 (q 2 (i 11 (q 2 (i (= 5 (point_add 11 (pubkey_for_exp (sha256 11 (a 6 (c 2 (c 23 ()))))))) (q 2 23 47) (q 8)) 1) (q 4 (c 4 (c 5 (c (a 6 (c 2 (c 23 ()))) ()))) (a 23 47))) 1) (c (q 50 2 (i (l 5) (q 11 (q . 2) (a 6 (c 2 (c 9 ()))) (a 6 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q . 0x93c8a14b74d0cfe62cf451200eb42f3f2fe5c581f5153d308b10c43f40fba9d49a812204e0f970b2f83e23c59eb22e09) 1))
```

##### Synthetic PK
`0x93c8a14b74d0cfe62cf451200eb42f3f2fe5c581f5153d308b10c43f40fba9d49a812204e0f970b2f83e23c59eb22e09`

#### RECOVERY_DID_LIST_HASH
`0x4bf5122f344554c53bde2ebb8cd2b7e3d1600ad631c385a5d7cce23c7785459a`

#### NUM_VERIFICATIONS_REQUIRED 
`()` 

#### SINGLETON_STRUCT
`(0x7faa3253bfddd1e0decb0906b2dc6247bbc4cf608f58345d173adb63e8b47c9f 0x7dc07df2dd088ffdfcbe5a273825ac55565e536ed06604df55b3afa67e44b107 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9)`

#### METADATA
`()`

#### Solution of singlton top layer
`lineage_proof my_amount inner_solution`
``` clojure
(
    (0x389e85e5bc97d408d0aad44e754490fccb047326361697173732e42f8cb6300f 1) ; lineage_proof
    1 ; my_amount
    (   q ; mode 1 
        (
            () 
            (   q 
                (
                 51 ; CREATE_COIN
                 0x3b28694e28afea944de7a5af588685ad7a30887f97cdebf719fc8b368dae85c6 ; did_innerpuz
                 1 ; did amount
                 (0xd34d627dec07b56ed99319d53d5ae1340e5bc8f262216c946abf79764a9c6562) ; memo (wallet ph)
                )
            ) 
            ()
        ) ; my_amount_or_inner_solution
    ) ; inner_solution
)

```

In [34]:
curried_standard_puzzle = curried_args.first()
print_program(curried_standard_puzzle)
print()
result = curried_standard_puzzle.uncurry()
standard_puzzle = result[0]
print_program(standard_puzzle)
print()
print_program(result[1])

(a (q 2 (q 2 (i 11 (q 2 (i (= 5 (point_add 11 (pubkey_for_exp (sha256 11 (a 6 (c 2 (c 23 ()))))))) (q 2 23 47) (q 8)) 1) (q 4 (c 4 (c 5 (c (a 6 (c 2 (c 23 ()))) ()))) (a 23 47))) 1) (c (q 50 2 (i (l 5) (q 11 (q . 2) (a 6 (c 2 (c 9 ()))) (a 6 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q . 0x93c8a14b74d0cfe62cf451200eb42f3f2fe5c581f5153d308b10c43f40fba9d49a812204e0f970b2f83e23c59eb22e09) 1))

(a (q 2 (i 11 (q 2 (i (= 5 (point_add 11 (pubkey_for_exp (sha256 11 (a 6 (c 2 (c 23 ()))))))) (q 2 23 47) (q 8)) 1) (q 4 (c 4 (c 5 (c (a 6 (c 2 (c 23 ()))) ()))) (a 23 47))) 1) (c (q 50 2 (i (l 5) (q 11 (q . 2) (a 6 (c 2 (c 9 ()))) (a 6 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1))

(0x93c8a14b74d0cfe62cf451200eb42f3f2fe5c581f5153d308b10c43f40fba9d49a812204e0f970b2f83e23c59eb22e09)


`did_inner_puzzle_hash`: `0x3b28694e28afea944de7a5af588685ad7a30887f97cdebf719fc8b368dae85c6`

`wallet_address_1`: `txch16dxkyl0vq76kakvnr82n6khpxs89hj8jvgske9r2hauhvj5uv43q9mcaa0`

`wallet_ph_1`: `d34d627dec07b56ed99319d53d5ae1340e5bc8f262216c946abf79764a9c6562`

In [44]:
%%bash
chia keys derive -f 3855208971 child-key -t wallet -n 5
chia keys derive -f 3855208971 wallet-address -n 5

Wallet public key 0: b87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84
Wallet public key 1: 97c6c9f4e251dd874156c2b49344285258e45f9e10797612b90baecef8da69b3fda76f9889ff97a90fc82cc8dfde7f99
Wallet public key 2: 8febecc9e2cf0c7c63fe3d024dee1fe44dc92cba8643ad0c3dbb832396a41e1448b16d55f889341bd225894d8ef29e7d
Wallet public key 3: b2c81aed4a3d3926784c04b4520b2f4dd31bff899147a7ea246f33741aa70d5fc22e3c53fcf9bed775e90e81f4f7001b
Wallet public key 4: 8795eaebf0c8f7755fe71baab1dbc49a47f64131c5980091cea56525e439744d75a4293e89034038a781e280215e61f1
Wallet address 0: txch1s705m30vt5cvdd4wex59x8fwcgym6cvd49n6ffrzmm8vlyvycansfvjfp9
Wallet address 1: txch16dxkyl0vq76kakvnr82n6khpxs89hj8jvgske9r2hauhvj5uv43q9mcaa0
Wallet address 2: txch1zczw8x349yaadzeesmjuxyg4a0mu9clysam90xv267a0gvauz6sswg35x3
Wallet address 3: txch19q9pef7caahchaa2ermpkfa4xxdxp6929n2nanx4p35ep2p5408s0jzvgm
Wallet address 4: txch1rdueaunf6jaqunz6y0hn332dz78f49exem5hzcf3nd32d7jajads9zce69


In [49]:
wallet_pk_1 = G1Element.from_bytes(bytes.fromhex("97c6c9f4e251dd874156c2b49344285258e45f9e10797612b90baecef8da69b3fda76f9889ff97a90fc82cc8dfde7f99"))
curreied_standard_puzzle = puzzle_for_pk(wallet_pk_1)
alice_ph_1 = curreied_standard_puzzle.get_tree_hash()
alice_addr_1 = encode_puzzle_hash(alice_ph, "txch")
print(f"alice_ph_1:\t{alice_ph_1}")
print(f"alice_addr_1:\t{alice_addr_1}")

synthetic_pk = calculate_synthetic_public_key(wallet_pk_1, DEFAULT_HIDDEN_PUZZLE_HASH)
print(f"synthetic_pk:\t{synthetic_pk}")

alice_ph_1:	d34d627dec07b56ed99319d53d5ae1340e5bc8f262216c946abf79764a9c6562
alice_addr_1:	txch16dxkyl0vq76kakvnr82n6khpxs89hj8jvgske9r2hauhvj5uv43q9mcaa0
synthetic_pk:	93c8a14b74d0cfe62cf451200eb42f3f2fe5c581f5153d308b10c43f40fba9d49a812204e0f970b2f83e23c59eb22e09


### Conclusion
`did:chia:10hq8mukapz8lml97tgnnsfdv24t9u5mw6pnqfh64kwh6vljykyrsc4m38t` belongs to Alice, because the `synthetic_pk` matches!